In [1]:
import pandas as pd
import reverse_geocoder as rg
from geopy.geocoders import Nominatim 
from googletrans import Translator, constants
translator = Translator()
geolocator = Nominatim(user_agent="geoapiExercises")

# Data cleaning & merging

In [2]:
def input_org_data():
    e_data = pd.read_csv("https://www.dropbox.com/s/u2xupuhf07y85zh/earthQuake.csv?dl=1")
    density_data = pd.read_csv("https://www.dropbox.com/s/p6ejr4mzx8pjq9m/population_density_long.csv?dl=1")
    population_data=pd.read_csv("https://www.dropbox.com/s/bp5hb61zrui2qo6/population_total_long.csv?dl=1")

In [3]:
e_data_with_country = pd.read_csv('out_e_data.csv', delimiter=',', encoding="utf-8")
density_data = pd.read_csv('population_density_long.csv', delimiter=',', encoding="utf-8")
population_data=pd.read_csv('population_total_long.csv', delimiter=',', encoding="utf-8")

In [4]:
full_combined_dataset = pd.read_csv('pop_edata.csv', delimiter=',', encoding="utf-8")
print(full_combined_dataset)

FileNotFoundError: [Errno 2] No such file or directory: 'pop_edata.csv'

In [ ]:
def cleanEarthquakeData(): 
    e_data[['Month','Day','Year']]=e_data['Date'].str.split('/',-1,expand=True)
    e_data = e_data.drop(['Date','Time','ID','Source','Location Source','Magnitude Source','Status','Type','Magnitude Type'],axis=1)
    print(e_data)

In [ ]:
def clean_combine_pop_density():
    population_data = population_data.drop(population_data.index[population_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
    population_data.rename(columns={'Count': 'Population'}, inplace=True)

    density_data = density_data.drop(density_data.index[density_data['Year'].isin(['1960', '1961', '1962','1963','1964'])])
    density_data.rename(columns={'Count': 'Density'}, inplace=True)

    joined = density_data.merge(population_data,
              how='left',
              on =['Country Name', 'Year'],
             )
    joined['Population'] = joined['Population'].fillna(0).astype(int)
    print(joined)
    pop_dense_data = joined
    #  suffixes=['', '_r']).drop(columns = ['date_r', 'year-month']


In [ ]:
def find_country(long,lat):
    tuple_input = (long,lat)
    found = rg.search(tuple_input)
    countryName = found[0]['name']
#     countryName = translator.translate(countryName)
    print('**', countryName)
    
    return countryName

def city_to_country(city):
    try:
        loc = geolocator.geocode(city, language='en')
        loc_list = loc.address.split(',')
    except:
        print(city + ': Not found')
        return None
    return loc_list[-1]

In [ ]:
def country_col_edata():
    count = 0
    for i, r, in e_data.iterrows():
        country = city_to_country(find_country(r['Latitude'], r['Longitude']))
        e_data['Country Name'][count] = country
        print(country)
        count +=1
    
    e_data_with_country = e_data_with_country.dropna()
    print(e_data_with_country)
    return e_data_with_country

In [ ]:
def strip_white_space():
    fixed_e_data = e_data_with_country
    fixed_e_data['Country Name'] = fixed_e_data['Country Name'].str.strip()
    g = fixed_e_data['Country Name'][2]
    print(type(g))
    print(g)

def merge_popDense_e_data():
    joined = e_data_with_country.merge(fixed_pop_data,
           how='inner',
          on =['Country Name', 'Year'],
         )
    joined['Population'] = joined['Population'].fillna(0).astype(int)
    joined['Density'] = joined['Density'].fillna(0).astype(int)
    print(joined)


In [7]:
#Clean data
e_data = pd.read_csv("https://www.dropbox.com/s/ey2y94hyfclgl98/clean_data.csv?dl=1")